# Introduction

We will start with a discussion on APIs, then we will setup our Jupyter notebook, and then we conquer the LLMs.

# Application Programming Interfaces

An Application Programming Interface or API is a mechanisms that enables two software components to communicate with each other using a set of definitions and protocols. ([AWS](https://aws.amazon.com/what-is/api/))

+ An *application* is any software with a distinct function. 
+ An *interface* can be seen as a contract between two applications that specifies how they will communicate with each other.

![](./img/01_api.svg)

## Types of APIs

There are four ways API can work:

### SOAP APIs

+ Simple Object Access Protocol. 
+ Client and server exchange using XML. 
+ Popular in the past, but less flexible than more modern alternatives.

### RPC APIs

+ Remote Procedural Calls.
+ Client completes a function (or procedure) on the server. 
+ The server sends the output back to the client.

### Websocket APIs

+ Uses JSON objects to pass data.
+ Supports two-way communication between client app and server.
+ Server can send callback messages to connected clients, making it more efficient than REST API.

### REST APIs

+ Representational State Transfer.
+ Most popular and flexible APIs found today.
+ Client sends request to the server as data.
+ Server uses this client input to start internal functions and returns output data back to the client.
+ Defines a set of functions like GET, PUT, DELETE, etc. that clients can use to access server data.
+ Clients and servers exchange data using HTTP.
+ REST APIs are stateless: servers do not save client data between requests. 

## API Endpoints

API endpoints are the final touchpoints in the API communication system. API endpoints can be server URLs, services, and other specific digital locations where information is sent and received between systems.

Two critical aspects about API Endpoints are:

1. Security: API endpoints make the system vulnerable to attack.
2. Performance: API endpoints, especially high traffic ones, can cause bottelnecks.

## OpenAI's API

+ The API provided by OpenAI is a useful starting point for building AI applications.
+ The API provides endpoints for various services, for example:

    - Responses API: https://api.openai.com/v1/responses
    - Conversations API: https://api.openai.com/v1/conversations
    - Videos API: https://api.openai.com/v1/videos
    - Embeddings API: https://api.openai.com/v1/embeddings
    - Eval API: https://api.openai.com/v1/evals

+ As well, OpenAI offers [Software Development Kits (SDK)](https://platform.openai.com/docs/libraries#install-an-official-sdk) for their APIs. These SDKs, allow us to interact with the API with Python functions instead of forming URLs and using tools like curl. SDKs are available for Python, JavaScript, .NET, Java, and Go.
+ The API is not the only interface to OpenAI's models and services, for example, 

    - Web apps are used to interact with GPT models via a chat client, [ChatGPT](https://chatgpt.com/)).
    - Developers can create agentic workflows using [Agent Builder](https://platform.openai.com/agent-builder) a no-code/low-code alternative to the API.

# Authentication

+ Authentication is the process of verifying the identity of a user or system.
+ Authentication (who you are) is generally paired with authorization (what you can do). 
+ Authenticating to the OpenAI service is done through an SSH Key, Secret Key, or API Key.
+ Details can be found in [OpenAI's API Documentation](https://platform.openai.com/docs/api-reference/introduction).


## Obtaining and Using API Keys

+ You can obtain OpenAI API Keys from [this page](https://platform.openai.com/api-keys).
+ Consider the following [Best Practices for API Key Safety](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety):

    1. Always use a unique API key for each team member on your account. 
    2. Never deploy your key in client-side environments like browsers or mobile apps.
    3. Never commit your key to your repository.
    4. Use Environment Variables in place of your API key.
    5. Use a Key Management Service.
    6. Monitor your account usage and rotate your keys when needed.
    


# Jupyter Notebook Setup

In this section we discuss a few preliminaries that will be useful throughout our lab sessions.



## Update System Path

+ Add the folder `./05_src/` to the system path:
    
    - This allows us to reutilize our modules in this notebook.
    - We can also avoid duplication as we build on top the code that we have written before.

+ The next cell imports the sys module and appends a relative path ('../../05_src/') to the system path.
+ This allows Python to locate and import custom modules from that directory in subsequent cells.

In [1]:
import sys
sys.path.append('../../05_src/')

## Use a Logger

- A logger affords observability and retains your logs.
- Log formats are customizable and you can include items like timestamp, module, function, line number, and so on.
- Log level is also customizable:

    + INFO for regular operations.
    + DEBUG for development.
    + ERROR and WARNING will be logged.

- Useful documents on logging:

    - [Python logging library](https://docs.python.org/3/library/logging.html).
    - [Real Python: Logging](https://realpython.com/python-logging/).
    - [The Hitchhiker's Guide to Python: Logging](https://docs.python-guide.org/writing/logging/).

The code cell below imports the function `get_logger()` from the module `utils.logger`. Notice that this module is located in ./05_src/utils/logger.py. We can directly load the module because we added the source folder (05_src) to our system path above.

In [2]:
from utils.logger import get_logger
_logs = get_logger(__name__, log_dir='../../06_logs/')

In [3]:
_logs.info('This is a log message.')

2025-10-13 15:33:53,060, 999434666.py, 1, INFO, This is a log message.


## Loading Environment Variables

+ Environment variables are stored in the operating system environment and not declared within the application itself.
+ They are convenient vairables to store settings such as file locations, directories, operational parameters, and log levels, among others.
+ They can also be used to store secrets (API keys, passwords, etc.)

### Dotenv and .env

We can set environment variables in the terminal window, but we can also use a convenient library called [`python-dotenv`](https://pypi.org/project/python-dotenv/).

From a Python module, you can call this functionality as below.

In [5]:
from dotenv import load_dotenv
load_dotenv('../../05_src/.secrets')

True

However, from a Jupyter notebook you would usually use something like:

In [7]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


We can obtain the the value of an enviornment variable using [`os.getenv()`](https://docs.python.org/3/library/os.html#os.getenv).

In [8]:
import os
os.getenv('LOG_LEVEL')

'DEBUG'

### About .secrets

The file .secrets is similar to .env in the sense that it contains key-value pairs meant to be set as environment variables. However, we seggregate certain variables, the secrets, into a special file which is then ignored by .git by adding it to .gitignore.

A sample of the expected format of .secrets is .secrets.template.

# Hello World!

We will use the [OpenAI Python API library](https://github.com/openai/openai-python?tab=readme-ov-file) as our main tool to communicate with OpenAI's API.

The code cell below makes a first call to the Responses API.

In [10]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model = 'gpt-4o',
    input = 'Hello world!',
)
print(response.output_text)

Hello! How are you today?


There are several things happening in this code:

1. Load the OpenAI library and instantiate a client object. The client object handles authentication, API calls, request/response handling, and error handling. In particular, it will look for an API key in an environment variable called `OPENAI_API_KEY`.

In [ ]:
from openai import OpenAI
client = OpenAI()

2. We create an API call and store the result in the variable `response`. Notice that the call specifies the model that we want to use, as well as an input. This is a simple call, the [responses API can handle more complex calls](https://platform.openai.com/docs/api-reference/responses).

In [ ]:
response = client.responses.create(
    model = 'gpt-4o',
    input = 'Hello world!',
)

3. Print out `output_text` from the response. The repsonse object will contain an attribute called `output` (a list) that contains content (another list) and the content contains text. Below we show these relationships.

In [21]:
print(response.output[0].content[0].text)

Hello! How are you today?


In the sample code, we used a convenience attribute called `output_text` that includes a concatenation of the text in all content and all output.

In [22]:
print(response.output_text)

Hello! How are you today?


Finally, we show the JSON-serialized version of the response object. The response object offers two methods to obtain JSON and dictionary versions of the repsonse: `repsonse.to_json()` and `response.to_dict()`.

In [14]:
print(response.to_json())

{
  "id": "resp_01b12c63540305620068ed5b5dbab08197b1d85bea92f1ab37",
  "created_at": 1760385885.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-2024-08-06",
  "object": "response",
  "output": [
    {
      "id": "msg_01b12c63540305620068ed5b5f38d481978253be0be3f8103d",
      "content": [
        {
          "annotations": [],
          "text": "Hello! How are you today?",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": null,
    "summary": null
  },
  "safety_identifier": null,
  "service_tier": "default",
  "status": "completed",
 

# Longer Context

- Let's try to add more context to our prompt.
- Python's [request](https://pypi.org/project/requests/) library and [documentation](https://requests.readthedocs.io/en/latest/).


In [ ]:
import requests
file_url = 'https://www.gutenberg.org/cache/epub/204/pg204.txt'
book = requests.get(file_url)

In [ ]:
dict(book.headers)

In [ ]:
print(book.text)

In [ ]:
prompt = f"""
    You are a literary historian. 
    Given the following context from a book, please summarize it in a concise manner. 
    Use no more than 4 sentences. 
    The book is the following: \n {book.text}
"""

In [ ]:
response = client.responses.create(
    model = 'gpt-4o',
    input = prompt,
)
response.output[0].content[0].text

In [ ]:
print(response.output[0].content[0].text)

**Note**: From the [documentation](https://platform.openai.com/docs/guides/text?api-mode=responses) we know that,

> The output array often has more than one item in it! It can contain tool calls, data about reasoning tokens generated by reasoning models, and other items. It is not safe to assume that the model's text output is present at output[0].content[0].text.
>
> Some of our official SDKs include an output_text property on model responses for convenience, which aggregates all text outputs from the model into a single string. This may be useful as a shortcut to access text output from the model.

In [ ]:
print(response.output_text)

# Adding a System Prompt

In [ ]:
system_prompt = "You are an anthropologist specializing in ancient fables."

In [ ]:
the_ant = """
Ants were once men and made their living by tilling the soil. 
But, not content with the results of their own work, they were 
always casting longing eyes upon the crops and fruits of their 
neighbours, which they stole, whenever they got the chance, and 
added to their own store. At last their covetousness made Jupiter 
so angry that he changed them into Ants. But, though their forms 
were changed, their nature remained the same: and so, to this day, 
they go about among the cornfields and gather the fruits of others’ 
labour, and store them up for their own use.
"""

prompt = f"""01_introduction.ipynb
    Please, extract the main message of this fable written by Aesop.
    The fable is the following: \n {the_ant}
"""

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    instructions = system_prompt,
    input = prompt,
)

print(response.output_text)

# APIs and SDK

+ OpenAI's Python library is an interface between our code written in Python and a RESTful API provided by OpenAI, a model provider.
+ These type of interfaces are sometimes called Software Development Kit (SDK).
+ You can read more in OpenAI's Documentation. A few good places to start are:

    - Python library's [Github repo](https://github.com/openai/openai-python?tab=readme-ov-file) has a good quickstart.
    - [Responses API](https://platform.openai.com/docs/api-reference/responses/) reference. 

## About the API

From the [documentation](https://platform.openai.com/docs/api-reference/responses):

> OpenAI's most advanced interface for generating model responses. Supports text and image inputs, and text outputs. Create stateful interactions with the model, using the output of previous responses as input. Extend the model's capabilities with built-in tools for file search, web search, computer use, and more. Allow the model access to external systems and data using function calling.


## Abou the Python Library

In summary, we used some form of the code below:

```{python}

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.responses.create(
    model = <a model>,
    instructions = <a system prompt>,
    input = <a user prompt>,
)

print(response.output_text)


```